In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [7]:
from skfeature.function.similarity_based import fisher_score

result= fisher_score.fisher_score(X_sm_tl, y_sm_tl)
result

C:\Users\User\anaconda3\lib\site-packages\skfeature\utility\construct_W.py:194: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  class_idx_all = class_idx[:, np.newaxis] & class_idx[np.newaxis, :]


array([ 1, 19,  2,  3, 14,  7,  4, 11, 13, 16, 17,  8, 21, 15,  0,  6,  5,
       20, 10, 12,  9, 18])

In [8]:
k= 10
top_k_indices= result[:k]

feature_names= X.columns
fisher_df= pd.DataFrame({'Feature': feature_names,
                         'Fisher Score': [np.nan] * len(feature_names)})

for idx in range(len(result)):
    fisher_df.loc[result[idx], 'Fisher Score']= idx + 1
    
fisher_df= fisher_df.sort_values(by='Fisher Score')
fisher_df.head(10)

,Feature,Fisher Score
1,Sex,1.0
19,Sleep Hours Per Day,2.0
2,Cholesterol,3.0
3,Heart Rate,4.0
14,Sedentary Hours Per Day,5.0
7,Obesity,6.0
4,Diabetes,7.0
11,Previous Heart Problems,8.0
13,Stress Level,9.0
16,BMI,10.0


In [9]:
X_sm_tl_selected= X_sm_tl[:, top_k_indices]
X_sm_tl_test_selected= X_test[:, top_k_indices]

--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.48      0.55      1125
           1       0.35      0.50      0.41       628

    accuracy                           0.49      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.3515625
Accuracy: 0.4900171135196805
AUC: 0.4944331210191083


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62      1125
           1       0.35      0.37      0.36       628

    accuracy                           0.52      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.52      0.53      1753

Precision: 0.34711964549483015
Accuracy: 0.5236737022247575
AUC: 0.490657466383581


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.6578288425706961


In [13]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.64      0.86      0.73      1125
           1       0.36      0.15      0.21       628

    accuracy                           0.60      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.60      0.55      1753

Precision: 0.3607843137254902
Accuracy: 0.6012549914432401
AUC: 0.5068131634819534


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.65      0.53      0.58      1125
           1       0.37      0.48      0.42       628

    accuracy                           0.51      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.51      0.52      1753

Precision: 0.36550060313630883
Accuracy: 0.5145464917284654
AUC: 0.5153503184713376


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.6841254208001268


In [16]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.77      0.70      1125
           1       0.35      0.22      0.27       628

    accuracy                           0.57      1753
   macro avg       0.49      0.50      0.48      1753
weighted avg       0.54      0.57      0.55      1753

Precision: 0.3493670886075949
Accuracy: 0.573873359954364
AUC: 0.5071139419674452


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.65      0.52      0.58      1125
           1       0.37      0.50      0.42       628

    accuracy                           0.51      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.51      0.52      1753

Precision: 0.3689776733254994
Accuracy: 0.5145464917284654
AUC: 0.49910969568294405


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.64      0.70      0.67      1125
           1       0.35      0.29      0.31       628

    accuracy                           0.55      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.55      0.54      1753

Precision: 0.34689922480620156
Accuracy: 0.551625784369652
AUC: 0.5090106157112526


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.65      0.84      0.73      1125
           1       0.39      0.19      0.25       628

    accuracy                           0.60      1753
   macro avg       0.52      0.51      0.49      1753
weighted avg       0.56      0.60      0.56      1753

Precision: 0.3920265780730897
Accuracy: 0.6046776953793497
AUC: 0.5093949044585988


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Precision Score: 0.7575320710288285


In [22]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.64      0.86      0.73      1125
           1       0.36      0.14      0.20       628

    accuracy                           0.60      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.60      0.54      1753

Precision: 0.3588709677419355
Accuracy: 0.6018254420992584
AUC: 0.5025859872611466


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.64      0.69      0.66      1125
           1       0.36      0.31      0.33       628

    accuracy                           0.55      1753
   macro avg       0.50      0.50      0.50      1753
weighted avg       0.54      0.55      0.54      1753

Precision: 0.35753176043557167
Accuracy: 0.5521962350256703
AUC: 0.5020552016985138


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6629573929204182


In [26]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.63      0.86      0.73      1125
           1       0.29      0.10      0.15       628

    accuracy                           0.59      1753
   macro avg       0.46      0.48      0.44      1753
weighted avg       0.51      0.59      0.52      1753

Precision: 0.2927927927927928
Accuracy: 0.5892755276668569
AUC: 0.5137664543524416


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.34      0.02      0.04       628

    accuracy                           0.64      1753
   macro avg       0.49      0.50      0.41      1753
weighted avg       0.53      0.64      0.51      1753

Precision: 0.34285714285714286
Accuracy: 0.6354820308043354
AUC: 0.5003326256192497


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7309396169816548


In [29]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.96      0.77      1125
           1       0.41      0.05      0.09       628

    accuracy                           0.63      1753
   macro avg       0.53      0.51      0.43      1753
weighted avg       0.56      0.63      0.53      1753

Precision: 0.41025641025641024
Accuracy: 0.6337706788362807
AUC: 0.5183170559094126


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.64      0.78      0.70      1125
           1       0.36      0.23      0.28       628

    accuracy                           0.58      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.58      0.55      1753

Precision: 0.3641025641025641
Accuracy: 0.5812892184826013
AUC: 0.5058853503184714


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7134619244643388
              precision    recall  f1-score   support

           0       0.65      0.89      0.75      1125
           1       0.41      0.13      0.20       628

    accuracy                           0.62      1753
   macro avg       0.53      0.51      0.47      1753
weighted avg       0.56      0.62      0.55      1753

Precision: 0.40594059405940597
Accuracy: 0.6200798630918426
AUC: 0.5006171266808209


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000344 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2212
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.65      0.90      0.75      1125
           1       0.41      0.12      0.18       628

    accuracy                           0.62      1753
   macro avg       0.53      0.51      0.47      1753
weighted avg       0.56      0.62      0.55      1753

Precision: 0.40555555555555556
Accuracy: 0.6223616657159156
AUC: 0.5120481245576787


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2212
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.794790268262178
[LightGBM] [Warning] min_data_in_leaf is set=30, min_

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.02579
0:	learn: 0.6908412	total: 153ms	remaining: 2m 32s
1:	learn: 0.6871830	total: 157ms	remaining: 1m 18s
2:	learn: 0.6853101	total: 162ms	remaining: 53.8s
3:	learn: 0.6825108	total: 167ms	remaining: 41.5s
4:	learn: 0.6800270	total: 171ms	remaining: 34.1s
5:	learn: 0.6768957	total: 176ms	remaining: 29.1s
6:	learn: 0.6740049	total: 180ms	remaining: 25.6s
7:	learn: 0.6714804	total: 185ms	remaining: 22.9s
8:	learn: 0.6699407	total: 190ms	remaining: 21s
9:	learn: 0.6668817	total: 196ms	remaining: 19.4s
10:	learn: 0.6656297	total: 201ms	remaining: 18.1s
11:	learn: 0.6633601	total: 205ms	remaining: 16.9s
12:	learn: 0.6616354	total: 210ms	remaining: 16s
13:	learn: 0.6580209	total: 215ms	remaining: 15.1s
14:	learn: 0.6555826	total: 219ms	remaining: 14.4s
15:	learn: 0.6536220	total: 224ms	remaining: 13.8s
16:	learn: 0.6512327	total: 228ms	remaining: 13.2s
17:	learn: 0.6487816	total: 233ms	remaining: 12.7s
18:	learn: 0.6469747	total: 238ms	remaining: 12.3s
19:	learn: 0.6

164:	learn: 0.5427844	total: 1.06s	remaining: 5.35s
165:	learn: 0.5426118	total: 1.06s	remaining: 5.35s
166:	learn: 0.5422846	total: 1.07s	remaining: 5.34s
167:	learn: 0.5420638	total: 1.08s	remaining: 5.34s
168:	learn: 0.5418357	total: 1.09s	remaining: 5.34s
169:	learn: 0.5415603	total: 1.09s	remaining: 5.34s
170:	learn: 0.5413365	total: 1.1s	remaining: 5.33s
171:	learn: 0.5409942	total: 1.1s	remaining: 5.32s
172:	learn: 0.5407909	total: 1.11s	remaining: 5.32s
173:	learn: 0.5405362	total: 1.12s	remaining: 5.31s
174:	learn: 0.5403320	total: 1.12s	remaining: 5.3s
175:	learn: 0.5400729	total: 1.13s	remaining: 5.29s
176:	learn: 0.5398963	total: 1.14s	remaining: 5.28s
177:	learn: 0.5397587	total: 1.14s	remaining: 5.28s
178:	learn: 0.5394720	total: 1.15s	remaining: 5.27s
179:	learn: 0.5392638	total: 1.15s	remaining: 5.26s
180:	learn: 0.5390826	total: 1.16s	remaining: 5.25s
181:	learn: 0.5385143	total: 1.16s	remaining: 5.24s
182:	learn: 0.5383272	total: 1.17s	remaining: 5.22s
183:	learn: 0.5

342:	learn: 0.5044501	total: 2.13s	remaining: 4.08s
343:	learn: 0.5042719	total: 2.13s	remaining: 4.07s
344:	learn: 0.5040330	total: 2.14s	remaining: 4.07s
345:	learn: 0.5037887	total: 2.15s	remaining: 4.06s
346:	learn: 0.5035434	total: 2.16s	remaining: 4.06s
347:	learn: 0.5033205	total: 2.16s	remaining: 4.05s
348:	learn: 0.5030787	total: 2.17s	remaining: 4.05s
349:	learn: 0.5027945	total: 2.18s	remaining: 4.04s
350:	learn: 0.5026259	total: 2.18s	remaining: 4.04s
351:	learn: 0.5021868	total: 2.19s	remaining: 4.03s
352:	learn: 0.5020112	total: 2.19s	remaining: 4.02s
353:	learn: 0.5018488	total: 2.2s	remaining: 4.02s
354:	learn: 0.5017047	total: 2.21s	remaining: 4.01s
355:	learn: 0.5015788	total: 2.21s	remaining: 4s
356:	learn: 0.5013913	total: 2.22s	remaining: 3.99s
357:	learn: 0.5012207	total: 2.22s	remaining: 3.98s
358:	learn: 0.5009045	total: 2.23s	remaining: 3.98s
359:	learn: 0.5007036	total: 2.23s	remaining: 3.97s
360:	learn: 0.5005082	total: 2.24s	remaining: 3.96s
361:	learn: 0.50

503:	learn: 0.4714012	total: 3.05s	remaining: 3s
504:	learn: 0.4711618	total: 3.06s	remaining: 3s
505:	learn: 0.4709588	total: 3.06s	remaining: 2.99s
506:	learn: 0.4707523	total: 3.07s	remaining: 2.98s
507:	learn: 0.4705637	total: 3.07s	remaining: 2.97s
508:	learn: 0.4704000	total: 3.08s	remaining: 2.97s
509:	learn: 0.4702557	total: 3.08s	remaining: 2.96s
510:	learn: 0.4700903	total: 3.08s	remaining: 2.95s
511:	learn: 0.4699557	total: 3.09s	remaining: 2.94s
512:	learn: 0.4697079	total: 3.09s	remaining: 2.94s
513:	learn: 0.4695302	total: 3.1s	remaining: 2.93s
514:	learn: 0.4694097	total: 3.1s	remaining: 2.92s
515:	learn: 0.4692875	total: 3.11s	remaining: 2.92s
516:	learn: 0.4691349	total: 3.11s	remaining: 2.91s
517:	learn: 0.4689748	total: 3.12s	remaining: 2.9s
518:	learn: 0.4688421	total: 3.12s	remaining: 2.89s
519:	learn: 0.4686858	total: 3.13s	remaining: 2.89s
520:	learn: 0.4685135	total: 3.13s	remaining: 2.88s
521:	learn: 0.4682405	total: 3.14s	remaining: 2.87s
522:	learn: 0.4679711

684:	learn: 0.4448738	total: 3.92s	remaining: 1.8s
685:	learn: 0.4447538	total: 3.92s	remaining: 1.79s
686:	learn: 0.4446276	total: 3.92s	remaining: 1.79s
687:	learn: 0.4444536	total: 3.93s	remaining: 1.78s
688:	learn: 0.4442991	total: 3.93s	remaining: 1.78s
689:	learn: 0.4441737	total: 3.94s	remaining: 1.77s
690:	learn: 0.4440211	total: 3.94s	remaining: 1.76s
691:	learn: 0.4439135	total: 3.95s	remaining: 1.76s
692:	learn: 0.4438104	total: 3.95s	remaining: 1.75s
693:	learn: 0.4436630	total: 3.96s	remaining: 1.75s
694:	learn: 0.4435049	total: 3.96s	remaining: 1.74s
695:	learn: 0.4433864	total: 3.97s	remaining: 1.73s
696:	learn: 0.4432786	total: 3.97s	remaining: 1.73s
697:	learn: 0.4431465	total: 3.98s	remaining: 1.72s
698:	learn: 0.4430218	total: 3.98s	remaining: 1.71s
699:	learn: 0.4429178	total: 3.99s	remaining: 1.71s
700:	learn: 0.4427935	total: 3.99s	remaining: 1.7s
701:	learn: 0.4426686	total: 4s	remaining: 1.7s
702:	learn: 0.4425520	total: 4s	remaining: 1.69s
703:	learn: 0.4424088

870:	learn: 0.4213033	total: 4.8s	remaining: 711ms
871:	learn: 0.4211567	total: 4.81s	remaining: 706ms
872:	learn: 0.4210577	total: 4.82s	remaining: 702ms
873:	learn: 0.4208898	total: 4.83s	remaining: 696ms
874:	learn: 0.4208011	total: 4.83s	remaining: 690ms
875:	learn: 0.4206531	total: 4.84s	remaining: 685ms
876:	learn: 0.4205451	total: 4.84s	remaining: 679ms
877:	learn: 0.4204611	total: 4.85s	remaining: 674ms
878:	learn: 0.4203767	total: 4.85s	remaining: 668ms
879:	learn: 0.4202909	total: 4.86s	remaining: 662ms
880:	learn: 0.4202013	total: 4.86s	remaining: 657ms
881:	learn: 0.4201070	total: 4.87s	remaining: 651ms
882:	learn: 0.4199852	total: 4.87s	remaining: 645ms
883:	learn: 0.4198395	total: 4.88s	remaining: 640ms
884:	learn: 0.4197021	total: 4.88s	remaining: 634ms
885:	learn: 0.4196482	total: 4.88s	remaining: 628ms
886:	learn: 0.4195133	total: 4.89s	remaining: 623ms
887:	learn: 0.4193989	total: 4.89s	remaining: 617ms
888:	learn: 0.4192583	total: 4.9s	remaining: 612ms
889:	learn: 0.

In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6892491	total: 2.62ms	remaining: 260ms
1:	learn: 0.6849747	total: 5.24ms	remaining: 257ms
2:	learn: 0.6823189	total: 7.74ms	remaining: 250ms
3:	learn: 0.6785031	total: 10.3ms	remaining: 247ms
4:	learn: 0.6753980	total: 13.2ms	remaining: 252ms
5:	learn: 0.6721053	total: 16ms	remaining: 251ms
6:	learn: 0.6702236	total: 18.7ms	remaining: 248ms
7:	learn: 0.6668057	total: 21.4ms	remaining: 246ms
8:	learn: 0.6645235	total: 24ms	remaining: 243ms
9:	learn: 0.6610300	total: 26.8ms	remaining: 241ms
10:	learn: 0.6582162	total: 29.6ms	remaining: 240ms
11:	learn: 0.6571978	total: 32.3ms	remaining: 237ms
12:	learn: 0.6553060	total: 35.3ms	remaining: 236ms
13:	learn: 0.6540275	total: 38.1ms	remaining: 234ms
14:	learn: 0.6512499	total: 41.1ms	remaining: 233ms
15:	learn: 0.6483610	total: 44.1ms	remaining: 232ms
16:	learn: 0.6458837	total: 46.9ms	remaining: 229ms
17:	learn: 0.6434970	total: 50ms	remaining: 228ms
18:	learn: 0.64188

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
12      Tuned GradientBoosting Precision:         0.410256
14                   Tuned XGB Precision:         0.405941
15                        LGBM Precision:         0.405556
7                Random Forest Precision:         0.392027
18              Tuned CatBoost Precision:         0.383562
5                   GaussianNB Precision:         0.368978
3         KNeighborsClassifier Precision:         0.365501
13                         XGB Precision:         0.364103
2          Tuned Decision Tree Precision:         0.360784
8          Tuned Random Forest Precision:         0.358871
9                     AdaBoost Precision:         0.357532
16                  Tuned LGBM Precision:         0.352941
0          Logistic Regression Precision:         0.351562
4   Tuned KNeighborsClassifier Precision:         0.349367
17                    CatBoost Precision:         0.349112
1                Decision Tree Precision:         0.3471

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
11            GradientBoosting Accuracy:        0.635482
12      Tuned GradientBoosting Accuracy:        0.633771
18              Tuned CatBoost Accuracy:        0.632059
16                  Tuned LGBM Accuracy:        0.630348
15                        LGBM Accuracy:        0.622362
14                   Tuned XGB Accuracy:        0.620080
17                    CatBoost Accuracy:        0.612664
7                Random Forest Accuracy:        0.604678
8          Tuned Random Forest Accuracy:        0.601825
2          Tuned Decision Tree Accuracy:        0.601255
10              Tuned AdaBoost Accuracy:        0.589276
13                         XGB Accuracy:        0.581289
4   Tuned KNeighborsClassifier Accuracy:        0.573873
9                     AdaBoost Accuracy:        0.552196
6                          SVM Accuracy:        0.551626
1                Decision Tree Accuracy:        0.523674
5                   GaussianNB 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
12      Tuned GradientBoosting AUC:   0.518317
3         KNeighborsClassifier AUC:   0.515350
10              Tuned AdaBoost AUC:   0.513766
15                        LGBM AUC:   0.512048
17                    CatBoost AUC:   0.511531
18              Tuned CatBoost AUC:   0.510201
7                Random Forest AUC:   0.509395
6                          SVM AUC:   0.509011
4   Tuned KNeighborsClassifier AUC:   0.507114
2          Tuned Decision Tree AUC:   0.506813
13                         XGB AUC:   0.505885
8          Tuned Random Forest AUC:   0.502586
9                     AdaBoost AUC:   0.502055
14                   Tuned XGB AUC:   0.500617
11            GradientBoosting AUC:   0.500333
5                   GaussianNB AUC:   0.499110
0          Logistic Regression AUC:   0.494433
16                  Tuned LGBM AUC:   0.492067
1                Decision Tree AUC:   0.490657
